## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import gkey
# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load = "../Weather_Database/WeatherPy_Database.csv"
city_data_df = pd.read_csv(file_to_load)
city_data_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,273.96,80,75,12.86,light snow
1,1,Vila Velha,BR,-20.3297,-40.2925,300.12,78,40,5.14,scattered clouds
2,2,Coquimbo,CL,-29.9533,-71.3436,287.00,88,0,2.06,clear sky
3,3,Hobart,AU,-42.8794,147.3294,286.51,74,75,2.24,light intensity drizzle
4,4,Jackson,US,42.4165,-122.8345,284.26,82,100,0.89,overcast clouds
5,5,Barrow,US,71.2906,-156.7887,268.69,86,100,3.45,overcast clouds
6,6,Albany,US,42.6001,-73.9662,284.15,84,96,0.93,overcast clouds
7,7,Kodiak,US,57.7900,-152.4072,279.49,85,57,3.23,broken clouds
8,8,Butaritari,KI,3.0707,172.7902,300.91,72,1,1.61,clear sky
9,9,Chulman,RU,56.8479,124.9104,269.96,86,100,1.57,overcast clouds


In [5]:
#Datatypes of dataframe.
city_data_df.dtypes

Unnamed: 0               int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vila Velha,BR,-20.3297,-40.2925,300.12,78,40,5.14,scattered clouds
8,8,Butaritari,KI,3.0707,172.7902,300.91,72,1,1.61,clear sky
14,14,Hithadhoo,MV,-0.6000,73.0833,301.50,69,21,7.00,few clouds
21,21,Isangel,VU,-19.5500,169.2667,301.21,69,75,2.57,broken clouds
31,31,Jatibarang,ID,-6.4747,108.3153,302.99,61,7,1.83,clear sky
39,39,Kapaa,US,22.0752,-159.3190,303.14,78,75,3.13,broken clouds
44,44,San Nicolas,PH,18.1725,120.5953,301.10,74,20,1.54,few clouds
47,47,Kirakira,SB,-10.4544,161.9205,302.78,70,22,4.63,few clouds
53,53,Cabo San Lucas,MX,22.8909,-109.9124,302.38,78,3,4.32,clear sky
59,59,Eydhafushi,MV,5.1033,73.0708,301.23,72,83,5.35,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
preferred_cities_df.count()

Unnamed: 0             107
City                   107
Country                107
Lat                    107
Lng                    107
Max Temp               107
Humidity               107
Cloudiness             107
Wind Speed             107
Current Description    107
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vila Velha,BR,300.12,scattered clouds,-20.3297,-40.2925,
8,Butaritari,KI,300.91,clear sky,3.0707,172.7902,
14,Hithadhoo,MV,301.50,few clouds,-0.6000,73.0833,
21,Isangel,VU,301.21,broken clouds,-19.5500,169.2667,
31,Jatibarang,ID,302.99,clear sky,-6.4747,108.3153,
39,Kapaa,US,303.14,broken clouds,22.0752,-159.3190,
44,San Nicolas,PH,301.10,few clouds,18.1725,120.5953,
47,Kirakira,SB,302.78,few clouds,-10.4544,161.9205,
53,Cabo San Lucas,MX,302.38,clear sky,22.8909,-109.9124,
59,Eydhafushi,MV,301.23,broken clouds,5.1033,73.0708,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   
   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    ## Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:  
       # r=hotels["results"][0]["name"] 
       # print("testing333",r)
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #hotels["results"][0]["name"]   
    except (IndexError):
        print("Hotel not found... skipping.")   
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [70]:
#Hotel Dataframe.
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Daru,PG,301.14,scattered clouds,-9.0763,143.2092,Daru Lodge
2,Cabo San Lucas,MX,305.16,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
21,Xapuri,BR,307.51,overcast clouds,-10.6517,-68.5044,Pousada dos Chapurys
33,Yeppoon,AU,300.03,clear sky,-23.1333,150.7333,Villa Mar Colina
43,Butaritari,KI,301.19,light rain,3.0707,172.7902,Isles Sunset Lodge
44,Ixtapa,MX,303.72,broken clouds,20.7000,-105.2000,Hotel Plaza Providencia
70,Estelle,US,302.62,broken clouds,29.8458,-90.1067,Boomtown Casino & Hotel New Orleans
71,Puerto Escondido,MX,300.61,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
74,Sao Filipe,CV,300.56,clear sky,14.8961,-24.4956,Hotel Xaguate
80,Hilo,US,303.31,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [15]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data into a csv
hotel_df.to_csv('WeatherPy_vacation.csv')

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dt>{Hotel Name}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country</b></dt><dt>{Country}</dt>
<dt><b>Current Description</b></dt><dt>{Current Description} and {Max Temp} °F</dt>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))